In [2]:
import numpy as np
import sys
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from scipy.integrate import quad
from scipy.optimize import minimize
from statsmodels.distributions.empirical_distribution import ECDF



def get_uv_from_xy(x,y):
    x_cdf = ECDF(x)
    y_cdf = ECDF(y)
    len_x = len(x)
    len_y = len(y)
    u, v = [len_x/(len_x+1)*x_cdf(a) for a in x], [len_y/(len_y+1)*y_cdf(a) for a in y]
    return u,v


def get_parameter(family, tau):
    """
    Estimate the theta parameter for the copula based on Kendall tau
    """
    if  family == 'clayton':
        return 2 * tau / (1 - tau)

    elif family == 'frank':
        integrand = lambda t: t / (np.exp(t) - 1)  # generate the integrand
        frank_fun = lambda theta: ((tau - 1) / 4.0  - (quad(integrand, sys.float_info.epsilon, theta)[0] / theta - 1) / theta) ** 2
        return minimize(frank_fun, 4, method='BFGS', tol=1e-5).x[0] 

    elif family == 'gumbel':
        return 1 / (1 - tau)


def pdf_copula(family, theta, u, v):
    """
    Estimate the probability density function of three kinds of Archimedean copulas
    """
    if  family == 'clayton':
        pdf = (theta+1) * ((u ** (-theta) + v ** (-theta) - 1) ** (-2 - 1/theta)) * (u ** (-theta-1) * v ** (-theta-1))

    elif family == 'frank':
        num = -theta * (np.exp(-theta) - 1) * (np.exp(-theta * (u + v)))
        denom = ((np.exp(-theta * u) - 1) * (np.exp(-theta * v) - 1) + (np.exp(-theta) - 1)) ** 2
        pdf = num / denom

    elif family == 'gumbel':
        A = (-np.log(u)) ** theta + (-np.log(v)) ** theta
        c = np.exp(-A ** (1 / theta))
        pdf = c * (u * v) ** (-1) * (A ** (-2 + 2/theta)) * ((np.log(u) * np.log(v)) ** (theta - 1)) * (1 + (theta - 1) * A ** (-1/theta))
    
    return pdf


def log_pdf(family, theta, u, v):
    pdf = pdf_copula(family, theta, u, v)
    return np.log(pdf)


def conditional_cdf(family,theta,u,v):
    """
    This is C(u|v) = dC/dv 
    Since u and v are symmetric, reverse the parameters for C(v|u)=dC/du
    """
    if family == 'clayton':
        ccdf =  v ** (-theta-1) * (u ** (-theta) + v ** (-theta) -1) ** (-1/theta -1)
            
    elif family == 'frank':
        exp_u = np.exp(-theta * u) - 1
        exp_v = np.exp(-theta * v) - 1
        ccdf = ( exp_u * exp_v + exp_u ) / ( exp_u * exp_v + (np.exp(-theta)-1) )

    elif family == 'gumbel':
        A = (-np.log(u)) ** theta + (-np.log(v)) ** theta
        c = np.exp(-A ** (1 / theta))
        ccdf = c * A ** ((1-theta)/theta) * (-np.log(v)) ** (theta-1) * (1/v)

    return ccdf


def add_copula_x_variable(df, df_price_1, df_price_2, train_start=None, train_end=None, ret_days=1):
    """
	Fit the copula on training data and create a new column with MI_u_v and MI_v_u
	"""

    # df.Date = pd.to_datetime(df.Date)
    if not train_start: train_start = df.Date.min()
    if not train_end: train_end = df.Date.max()
    # Compute total returns
    df_price_1 = df_price_1.pct_change(periods=ret_days)
    df_price_2 = df_price_2.pct_change(periods=ret_days)
    x_fit = df_price_1.loc[train_start : train_end+pd.Timedelta(days=1)].dropna()
    y_fit = df_price_2.loc[train_start : train_end+pd.Timedelta(days=1)].dropna()
    u_fit,v_fit = get_uv_from_xy(x_fit, y_fit)
    tau = kendalltau(x_fit, y_fit)[0]

    AIC ={}  # generate a dict with key being the copula family, value = [theta, AIC]
    for i in ['clayton', 'frank', 'gumbel']:
        param = get_parameter(i, tau)
        lpdf = [log_pdf(i, param, a, b) for (a, b) in zip(u_fit, v_fit)]
        lpdf = np.nan_to_num(lpdf) 
        loglikelihood = sum(lpdf)
        AIC[i] = [param, -2 * loglikelihood + 2]

    fitted_copula = min(AIC.items(), key = lambda a: a[1][1])[0]
    fitted_theta = AIC[fitted_copula][0]

    start_dt = df.Date.min()
    end_dt = df.Date.max()
    x = df_price_1.loc[start_dt : end_dt+pd.Timedelta(days=1)].dropna()
    y = df_price_2.loc[start_dt : end_dt+pd.Timedelta(days=1)].dropna()
    u,v = get_uv_from_xy(x, y)

    df_p_val = pd.DataFrame([u, v]).T
    df_p_val.index = x.index
    df_p_val.index.name = 'Date'
    df_p_val.columns = ['u', 'v']
    df_p_val['MI_u_v'] = df_p_val.apply(lambda r: conditional_cdf(fitted_copula, fitted_theta, r.u, r.v),axis=1)
    df_p_val['MI_v_u'] = df_p_val.apply(lambda r: conditional_cdf(fitted_copula, fitted_theta, r.v, r.u),axis=1)
    df_p_val.reset_index(inplace=True)
    

    df = df.merge(df_p_val[['Date','MI_u_v', 'MI_v_u']], on='Date', how='left')
    return df.set_index('Date')


In [ ]:
FEATURE_LIST_BASE = [
 'Div Yield',
 'Price to Book',
 'Price to Earnings',
 'Total_Ret_Pct_1',
 'Total_Ret_Pct_5',
 'Total_Ret_Pct_21',
 'Total_Ret_Pct_63',
 'Total_Ret_Pct_252',
 'Fast_MACD',
 'Med_MACD',
 'Slow_MACD',
 'Pair_RSI_10',
 'Pair_RSI_21',
 'Pair_RSI_63',
 'Bollinger_10',
 'Bollinger_21',
 'Bollinger_63',
 'zero_cross',
 'days_s_cross',
 'Real Credit Growth',
 'Liquidity Growth',
 'Change in Valuation',
 'IG Credit Spread',
 'HY Credit Spread',
 'TED Spread',
 'VIX_Roll_diff',
 'MI_u_v',
 'MI_v_u']

FEATURE_LIST = {
    'd': FEATURE_LIST_BASE + ['Y_1Fwd_Lagged_1','Y_1Fwd_Lagged_2','Y_1Fwd_Lagged_3','Y_1Fwd_Lagged_4','Y_1Fwd_Lagged_5'],
    'w': FEATURE_LIST_BASE + ['Y_5Fwd_Lagged_6','Y_5Fwd_Lagged_7','Y_5Fwd_Lagged_8','Y_5Fwd_Lagged_9','Y_5Fwd_Lagged_10'],
    'M': FEATURE_LIST_BASE + ['Y_21Fwd_Lagged_21', 'Y_5Fwd_Lagged_42', 'Y_5Fwd_Lagged_63', 'Y_5Fwd_Lagged_252']
}

In [3]:
# import __file__
import numpy as np
import pandas as pd
import pickle

import os
from typing import List, Tuple, Optional, Mapping
from tqdm import tqdm
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit,  GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from scipy import stats
from sklearn.metrics import make_scorer, r2_score



class Predictor():
    def __init__(self):
        pass
    
    def train(self, data: pd.DataFrame, params: Optional[Mapping] = None):
        pass

    def predict(self, data: pd.DataFrame, params: Optional[Mapping] = None) -> pd.DataFrame:
        pass

    def periodic_train_predict(self, data: pd.DataFrame, params: Optional[Mapping] = None) -> pd.DataFrame:
        pass


class XGBoostPredictor(Predictor):
    """
    Build XGBoost model
    """
    def __init__(self,freq,):
        super().__init__()
        self.model = None
        self.freq = freq
        self.features = FEATURE_LIST[freq]
        self.y_var = f'Y_Fwd_Total_Ret_Pct_{self.freq}'

    def train(self, data: pd.DataFrame, freq='d'):
        """
        
        """
        train_df = data.reset_index(drop=True)
        train_X = data[self.features]
        train_y = data[self.y_var]

        tscv = TimeSeriesSplit(n_splits=5)

        # param_grid = {
        #     'learning_rate': [0.1, 0.01,0.001],
        #     'max_depth': [1,2,3],
        #     'subsample': [0.7, 0.8, 0.9],
        #     'n_estimators': [50,100,150]
        # }
        param_grid = {
            # 'n_estimators': [100,200],
            'learning_rate': [0.1, 1],
            'max_depth': [1,2],
            'subsample': [0.8, 1],
            'colsample_bytree': [0.8, 1],
            'reg_lambda': [0, 1e-2, 1],
            'reg_alpha': [0, 1e-2, 1]
        }
        
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=10)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)

        grid_search.fit(train_X, train_y)
        best_params = grid_search.best_params_
        model_best = xgb.XGBRegressor(**best_params)
        model_best.fit(train_X, train_y)
        self.model = model_best


    def predict(self, data: pd.DataFrame, save_path: Optional[str] = None) \
        -> pd.DataFrame:
        """
        Day by day forecast
        """
        test_X = data[self.features]
        test_y = data[self.y_var]

        pred_y = self.model.predict(test_X)
        
        output_df = pd.DataFrame([pred_y, test_y]).T
        output_df.columns = ["pred_spread", "actual_spread"]
        output_df.index = data.index
        if save_path:
            output_df.to_pickle(save_path)
        return output_df

    def periodic_train_predict(self, data: pd.DataFrame, \
        save_path: Optional[str] = None) -> pd.DataFrame:
        pass


In [5]:
if __name__ == "__main__":
    freqs = ["d", "w", "M"]
    # modes = ["predict", "periodic_train_predict"]
    modes = ["predict"]
    
    # Set Path
    data_path = ""
    save_path = ""
    for mode in modes:
        if not os.path.exists(f"{save_path}{mode}/"):
            os.makedirs(f"{save_path}{mode}/")
        if not os.path.exists(f"{save_path}{mode}_models/"):
            os.makedirs(f"{save_path}{mode}_models/")

    # Load Pairs
    raw_price_df = pd.read_csv(data_path + "price_df.csv", parse_dates=["Date"])
    raw_price_df.sort_values(["ETF_Ticker", "Date"], inplace=True)
    feature_df_raw = pd.read_csv(data_path + "TrainingSet.csv", parse_dates=["Date"])
    feature_df_raw.sort_values(["Ticker_Pair", "Date"], inplace=True)
    pair_arr = np.unique(feature_df_raw["Ticker_Pair"].values)
    
    # Make predictions on each pair under 
    freq_map = {'d':1, 'w':5, 'M':21}
    for freq in tqdm(freqs):

        price_s = raw_price_df.set_index(["ETF_Ticker", "Date"]).squeeze()
        feature_df = feature_df_raw.set_index(["Ticker_Pair", "Date"]).squeeze()

        for mode in tqdm(modes):
            output_list = []
            for pair in pair_arr:
                print(pair)
                pair_list = pair.split("_")
                pair_s = price_s.loc[pair_list, :].copy()
                pair_df = pair_s.unstack("ETF_Ticker")
                pair_df.dropna(inplace=True)

                feature_df_pair = feature_df.loc[pair, :].copy()

                # Build Predictor
                predictor = XGBoostPredictor(freq_map[freq])
                if mode == "predict":
                    feature_df_pair = add_copula_x_variable(feature_df_pair.reset_index(), pair_df[pair_list[0]], pair_df[pair_list[1]],
                        train_end=pd.Timestamp('2016-12-31'),ret_days=freq_map[freq])
                    feature_df_pair['VIX_Roll_diff'] = feature_df_pair['VIX Roll'].diff()
                    train_df = feature_df_pair.loc[feature_df_pair.index < pd.Timestamp("2017-01-01")].copy().dropna()
                    test_df = feature_df_pair.loc[feature_df_pair.index >= pd.Timestamp("2017-01-01")].copy()
                    predictor.train(train_df)
                    pickle.dump(predictor.model, open(f"{save_path}{mode}_models/xgb_{freq}_{pair}.pkl", "wb"))

                    df_pred_train = predictor.predict(train_df)
                    df_pred_test = predictor.predict(test_df)
                    output_i_df = pd.concat([df_pred_train, df_pred_test])
               
                output_i_df.index.name = "Date"
                output_i_df.reset_index(inplace=True)
                output_i_df["pair"] = pair
                output_i_df = output_i_df.reindex(
                    ["pair"] + list(output_i_df.columns[:-1]), axis=1
                )
                output_list.append(output_i_df)
                
            output_df = pd.concat(output_list)
            output_df.to_pickle(f"{save_path}{mode}/ReturnSpreadPredictions_{freq}.pkl")

*XFN_IJH
Already there
*XIC_EZA
Already there
*XIU_REM
Already there
BKF_IGM
Already there
BKF_IVV
Already there
EWD_THD
Already there
EWG_THD
Already there
EWH_IJH
Already there
EWM_EZA
Already there
EWQ_IYG
Already there
EWT_IJH
Already there
EWT_IJJ
Already there
EWT_IVE
Already there
EWT_IVV
Already there
EWT_IWB
Already there
EWT_IWV
Already there
EZU_THD
Already there
IDU_IVV
Already there
IEV_THD
Already there
IFGL_IHE
IFGL_IWM
IGM_IHI
IHE_ACWI
IHE_ACWX
IHE_SCZ
IHE_TOK
IHE_WOOD
IHF_ILCB
IHF_ILCG
IHF_IUSG
IHF_IVW
IHF_IXN
IJH_ACWI
IJH_TOK
IJH_WOOD
IJJ_ACWI
IMCV_IWM
IVV_ACWI
IWB_ACWI
IWM_ACWI
IWM_SCZ
IWM_SUSA
IWM_WOOD
IWV_ACWI
IXP_IYH
IYH_ACWI
IYH_TOK
IYK_ACWI
REM_SUSA
RXI_WOOD


100%|██████████| 1/1 [1:38:57<00:00, 5937.39s/it]
